In [27]:
import pandas as pd
import requests
from scipy import stats
import math

In [28]:
stocks = pd.read_csv('sp_500_stocks.csv')
token = 'Tpk_059b97af715d417d9f49f50b51b1c448'
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [29]:
def chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 
           
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    

In [30]:
clm = ['Ticker', 'Price', 'Number of Shares to Buy', 'One Year Return',
        'One Year Return Percentile', 'Six month Return', 'Six month Return Percentile',
        'Three month Return', 'Three month Return Percentile', 'One month Return', 
        'One month Return Percentile','HQM Score']

final_data = pd.DataFrame(columns = clm)

for symbol_string in symbol_strings:
    batch_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={token}'
    data = requests.get(batch_url).json()
    for symbol in symbol_string.split(','):
        final_data = final_data.append(
        pd.Series(
            [   symbol,
                data[symbol]['price'],
                0,
                data[symbol]['stats']['year1ChangePercent'],
                0,
                data[symbol]['stats']['month6ChangePercent'],
                0,
                data[symbol]['stats']['month3ChangePercent'],
                0,
                data[symbol]['stats']['month1ChangePercent'],
                0,
                0
            ],
        index = clm),
        ignore_index = True)
    
final_data
    

,Ticker,Price,Number of Shares to Buy,One Year Return,One Year Return Percentile,Six month Return,Six month Return Percentile,Three month Return,Three month Return Percentile,One month Return,One month Return Percentile,HQM Score
0,A,126.71,0,0.723855,0,0.303033,0,0.0748421,0,0.0306466,0,0
1,AAL,22.99,0,0.474145,0,0.907221,0,0.470153,0,0.0952155,0,0
2,AAP,191.73,0,0.944285,0,0.265999,0,0.17538,0,0.170141,0,0
3,AAPL,124.23,0,0.917596,0,0.0861871,0,-0.0816623,0,-0.000426936,0,0
4,ABBV,110.45,0,0.532726,0,0.26985,0,0.0396474,0,-0.0164439,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.59,0,0.466928,0,0.198841,0,0.00979811,0,0.0442426,0,0
501,ZBH,161.57,0,0.627494,0,0.201686,0,0.0854789,0,-0.00941284,0,0
502,ZBRA,477.14,0,1.45968,0,0.880895,0,0.274501,0,-0.0458989,0,0
503,ZION,57.06,0,1.13308,0,1.0228,0,0.310633,0,0.0523292,0,0


In [31]:
time_periods = [
                'One Year',
                'Six month',
                'Three month',
                'One month'
                ]

for row in final_data.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Return'
        percentile_col = f'{time_period} Return Percentile'
        if final_data.loc[row, change_col] == None:
            final_data.loc[row, change_col] = 0.0
            
for row in final_data.index:
    for time_period in time_periods:
        change_col = f'{time_period} Return'
        percentile_col = f'{time_period} Return Percentile' 
        final_data.loc[row, percentile_col] = stats.percentileofscore(final_data[change_col], final_data.loc[row, change_col])/100
        
final_data

,Ticker,Price,Number of Shares to Buy,One Year Return,One Year Return Percentile,Six month Return,Six month Return Percentile,Three month Return,Three month Return Percentile,One month Return,One month Return Percentile,HQM Score
0,A,126.71,0,0.723855,0.631683,0.303033,0.520792,0.0748421,0.370297,0.0306466,0.289109,0
1,AAL,22.99,0,0.474145,0.386139,0.907221,0.934653,0.470153,0.976238,0.0952155,0.641584,0
2,AAP,191.73,0,0.944285,0.768317,0.265999,0.455446,0.17538,0.631683,0.170141,0.944554,0
3,AAPL,124.23,0,0.917596,0.756436,0.0861871,0.190099,-0.0816623,0.0356436,-0.000426936,0.158416,0
4,ABBV,110.45,0,0.532726,0.451485,0.26985,0.461386,0.0396474,0.257426,-0.0164439,0.10099,0
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.59,0,0.466928,0.380198,0.198841,0.368317,0.00979811,0.184158,0.0442426,0.360396,0
501,ZBH,161.57,0,0.627494,0.536634,0.201686,0.372277,0.0854789,0.405941,-0.00941284,0.128713,0
502,ZBRA,477.14,0,1.45968,0.936634,0.880895,0.920792,0.274501,0.837624,-0.0458989,0.0594059,0
503,ZION,57.06,0,1.13308,0.855446,1.0228,0.966337,0.310633,0.875248,0.0523292,0.4,0


In [32]:
for row in range(len(final_data[:1])):
    mean = (final_data['One Year Return Percentile'] + final_data['Six month Return Percentile'] 
    + final_data['Three month Return Percentile'] + final_data['One month Return Percentile'])/4
    final_data['HQM Score'] = mean
    
final_data

,Ticker,Price,Number of Shares to Buy,One Year Return,One Year Return Percentile,Six month Return,Six month Return Percentile,Three month Return,Three month Return Percentile,One month Return,One month Return Percentile,HQM Score
0,A,126.71,0,0.723855,0.631683,0.303033,0.520792,0.0748421,0.370297,0.0306466,0.289109,0.45297
1,AAL,22.99,0,0.474145,0.386139,0.907221,0.934653,0.470153,0.976238,0.0952155,0.641584,0.734653
2,AAP,191.73,0,0.944285,0.768317,0.265999,0.455446,0.17538,0.631683,0.170141,0.944554,0.7
3,AAPL,124.23,0,0.917596,0.756436,0.0861871,0.190099,-0.0816623,0.0356436,-0.000426936,0.158416,0.285149
4,ABBV,110.45,0,0.532726,0.451485,0.26985,0.461386,0.0396474,0.257426,-0.0164439,0.10099,0.317822
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.59,0,0.466928,0.380198,0.198841,0.368317,0.00979811,0.184158,0.0442426,0.360396,0.323267
501,ZBH,161.57,0,0.627494,0.536634,0.201686,0.372277,0.0854789,0.405941,-0.00941284,0.128713,0.360891
502,ZBRA,477.14,0,1.45968,0.936634,0.880895,0.920792,0.274501,0.837624,-0.0458989,0.0594059,0.688614
503,ZION,57.06,0,1.13308,0.855446,1.0228,0.966337,0.310633,0.875248,0.0523292,0.4,0.774257


In [33]:
final_data.sort_values('HQM Score', ascending = False, inplace = True)
final_data = final_data[:50]
final_data.reset_index(inplace = True)
final_data

,index,Ticker,Price,Number of Shares to Buy,One Year Return,One Year Return Percentile,Six month Return,Six month Return Percentile,Three month Return,Three month Return Percentile,One month Return,One month Return Percentile,HQM Score
0,208,GPS,29.640,0,2.57716,0.99604,0.829632,0.906931,0.450611,0.970297,0.188106,0.974257,0.961881
1,344,NUE,83.200,0,1.49435,0.938614,0.804961,0.89703,0.543428,0.992079,0.334744,0.99802,0.956436
2,275,LB,62.000,0,3.80784,0.99802,1.07227,0.970297,0.606189,0.99802,0.133193,0.853465,0.95495
3,352,OKE,53.280,0,1.38292,0.920792,1.21668,0.982178,0.355271,0.918812,0.158391,0.922772,0.936139
4,220,HIG,69.000,0,1.04606,0.819802,0.952377,0.952475,0.41939,0.958416,0.352219,1,0.932673
5,160,EOG,77.680,0,0.980213,0.786139,1.03816,0.968317,0.520984,0.988119,0.165104,0.936634,0.919802
6,462,URI,343.840,0,2.41777,0.992079,0.971057,0.956436,0.428647,0.962376,0.113746,0.752475,0.915842
7,175,FANG,77.430,0,2.08484,0.984158,1.6164,0.990099,0.600463,0.99604,0.103315,0.685149,0.913861
8,306,MHK,204.260,0,1.42692,0.924752,1.10761,0.974257,0.413017,0.950495,0.12134,0.79802,0.911881
9,253,IVZ,25.060,0,1.71286,0.960396,1.37082,0.986139,0.418805,0.954455,0.107957,0.714851,0.90396


In [35]:
def portfolioInput():
    global portfolio_size
    portfolio_size = float(input('Enter size of portfolio: '))
    
portfolioInput()

positionSize = float(portfolio_size/len(final_data))
for row in range(len(final_data)):
    final_data.loc[row, 'Number of Shares to Buy'] = math.floor(positionSize/final_data.loc[row,'Price'])
    
final_data

Enter size of portfolio: 100000


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,Price,Number of Shares to Buy,One Year Return,One Year Return Percentile,Six month Return,Six month Return Percentile,Three month Return,Three month Return Percentile,One month Return,One month Return Percentile,HQM Score
0,208,GPS,29.640,67,2.57716,0.99604,0.829632,0.906931,0.450611,0.970297,0.188106,0.974257,0.961881
1,344,NUE,83.200,24,1.49435,0.938614,0.804961,0.89703,0.543428,0.992079,0.334744,0.99802,0.956436
2,275,LB,62.000,32,3.80784,0.99802,1.07227,0.970297,0.606189,0.99802,0.133193,0.853465,0.95495
3,352,OKE,53.280,37,1.38292,0.920792,1.21668,0.982178,0.355271,0.918812,0.158391,0.922772,0.936139
4,220,HIG,69.000,28,1.04606,0.819802,0.952377,0.952475,0.41939,0.958416,0.352219,1,0.932673
5,160,EOG,77.680,25,0.980213,0.786139,1.03816,0.968317,0.520984,0.988119,0.165104,0.936634,0.919802
6,462,URI,343.840,5,2.41777,0.992079,0.971057,0.956436,0.428647,0.962376,0.113746,0.752475,0.915842
7,175,FANG,77.430,25,2.08484,0.984158,1.6164,0.990099,0.600463,0.99604,0.103315,0.685149,0.913861
8,306,MHK,204.260,9,1.42692,0.924752,1.10761,0.974257,0.413017,0.950495,0.12134,0.79802,0.911881
9,253,IVZ,25.060,79,1.71286,0.960396,1.37082,0.986139,0.418805,0.954455,0.107957,0.714851,0.90396
